In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from pymongo import MongoClient
import json

In [2]:
client = MongoClient('localhost', 27017)
mongodb = client.mvideo
collection = mongodb['hits']

In [3]:
chrome_options = Options()
chrome_options.add_argument('start-maximized')

In [4]:
driver = webdriver.Chrome(executable_path='./chromedriver', options=chrome_options)

In [5]:
driver.get('https://www.mvideo.ru/')

In [6]:
try:
    while True:
        elem = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.XPATH, '//div[@class="gallery-content accessories-new "][1]//a[contains(@class,"sel-hits-button-next")]')))
        if 'disabled' in elem.get_attribute('class'):
            break
        else:
            elem.click()
except:
    print('something went wrong in getting mvideo.ru')
elem = driver.find_element_by_xpath('//div[@class="gallery-content accessories-new "]')
items = elem.find_elements_by_class_name('gallery-list-item')
for item in items:
    item_data = json.loads(item.find_element_by_xpath('.//a[@data-product-info]').get_attribute('data-product-info'))
    collection.insert_one({'productId' : item_data['productId'], 
                           'productVendorName' : item_data['productVendorName'], 
                           'productName' : item_data['productName'], 
                           'productPriceLocal' : item_data['productPriceLocal']})
driver.quit()